# BiggerPockets ADU (Accessory Dwelling Units) Analysis

This notebook analyzes BiggerPockets forum posts to extract and analyze content related to ADUs (Accessory Dwelling Units).

In [22]:
import json
import pandas as pd
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn')
sns.set_palette("husl")

## 1. Load BiggerPockets Data

In [23]:
# Load the BiggerPockets data
data_path = '../data/raw/social-meida-analysis/biggerpockets.json'

with open(data_path, 'r', encoding='utf-8') as f:
    biggerpockets_data = json.load(f)

print(f"Total posts in file: {biggerpockets_data['metadata']['total_posts_in_file']}")
print(f"Number of cities: {len(biggerpockets_data['metadata']['cities_in_file'])}")
print(f"\nSample cities: {biggerpockets_data['metadata']['cities_in_file'][:10]}")

Total posts in file: 2295
Number of cities: 253

Sample cities: ['chattanooga-tennessee', 'augusta-georgia', 'kansas-city-kansas', 'tampa-florida', 'warren-michigan', 'cape-cod-massachusetts', 'el-paso-texas', 'blueridge-georgia', 'wilmington-delaware', 'bridgeport-connecticut']


## 2. Extract ADU-Related Posts

In [24]:
# Define ADU-related keywords and patterns
adu_keywords = [
    # Primary terms
    r'\bADU\b', r'\bADUs\b',
    r'accessory dwelling unit',
    r'accessory dwelling',
    
    # Common ADU types
    r'granny flat',
    r'mother-in-law suite',
    r'in-law suite',
    r'in-law unit',
    r'backyard cottage',
    r'garage conversion',
    r'garage apartment',
    r'basement apartment',
    r'secondary suite',
    r'secondary dwelling',
    r'casita',
    r'ohana unit',
    r'laneway house',
    r'coach house',
    r'carriage house',
    
    # Related terms
    r'converted garage',
    r'detached unit',
    r'attached unit',
    r'JADU',  # Junior ADU
    r'junior accessory dwelling',
    r'tiny house on property',
    r'backyard rental'
]

# Create a combined regex pattern
adu_pattern = '|'.join(adu_keywords)
adu_regex = re.compile(adu_pattern, re.IGNORECASE)

In [25]:
biggerpockets_data['posts'][10]

{'location': 'albuquerque-new-mexico',
 'title': 'Want to get back in the game. Looking for investment partners',
 'url': 'https://www.biggerpockets.com/forums/67/topics/1189580-want-to-get-back-in-the-game-looking-for-investment-partners',
 'topic_id': '1189580',
 'list_scraped_at': '2025-08-20T09:09:55.412544',
 'content': 'Hello BP community. I have been a real estate broker for 20 years now. I spent many years personal flipping homes and then most of my career helping investors buy and sell. I have been out of the game, personally, for too long and ready to get back. I am very intrigued by the BRRRR method but my skills and expertise still lie in fix-and-flips. I am looking for partners to help with the funding side of things. If you or anyone you know is interested in partnering in some deals to get back in the flipping game, I would love to chat.',
 'author': 'Randi Guthrie',
 'author_url': 'https://www.biggerpockets.com/users/randig',
 'post_date': '2024-05-10T09:19:26-07:00',
 

In [26]:
# Extract all posts and filter for ADU-related content
all_posts = []
adu_posts = []

# The data structure is actually a flat list of posts, not organized by city
# Each post has its own 'location' field
for post in biggerpockets_data['posts']:
    all_posts.append(post)
    
    # Check if post is ADU-related using both title and content
    text_to_search = f"{post.get('title', '')} {post.get('content', '')}"
    
    if adu_regex.search(text_to_search):
        # Find which keywords matched
        matched_keywords = []
        for keyword in adu_keywords:
            if re.search(keyword, text_to_search, re.IGNORECASE):
                # Clean up the keyword for display
                clean_keyword = keyword.replace(r'\b', '').replace('\\', '')
                matched_keywords.append(clean_keyword)
        
        # Create a copy of the post and add matched keywords
        adu_post = post.copy()
        adu_post['matched_keywords'] = matched_keywords
        adu_posts.append(adu_post)

print(f"Total posts analyzed: {len(all_posts)}")
print(f"ADU-related posts found: {len(adu_posts)}")
if len(all_posts) > 0:
    print(f"Percentage of ADU posts: {len(adu_posts)/len(all_posts)*100:.2f}%")

Total posts analyzed: 2295
ADU-related posts found: 31
Percentage of ADU posts: 1.35%


## 3. Convert to DataFrame for Analysis

In [27]:
# Convert ADU posts to DataFrame
if len(adu_posts) > 0:
    adu_df = pd.DataFrame(adu_posts)
    
    # Display basic information
    print("ADU Posts DataFrame Info:")
    print(f"Shape: {adu_df.shape}")
    print(f"\nColumns: {adu_df.columns.tolist()}")
    print(f"\nFirst few ADU posts:")
    display_cols = ['title', 'location', 'matched_keywords']
    # Check which columns actually exist
    display_cols = [col for col in display_cols if col in adu_df.columns]
    if len(display_cols) > 0:
        display(adu_df[display_cols].head())
else:
    print("No ADU posts found to create DataFrame")
    adu_df = pd.DataFrame()  # Create empty DataFrame

ADU Posts DataFrame Info:
Shape: (31, 17)

Columns: ['location', 'title', 'url', 'topic_id', 'list_scraped_at', 'content', 'author', 'author_url', 'post_date', 'view_count', 'vote_count', 'comment_count', 'replies', 'scraped_at', 'total_replies', 'scraped_from_city', 'matched_keywords']

First few ADU posts:


,title,location,matched_keywords
0,Selling Low Interest Rate Investment Property ...,boston-massachusetts,[ADU]
1,Learn About ADUs in Massachusetts,boston-massachusetts,"[ADU, ADUs]"
2,"How to Rent out ""Mixed-Use"" Zoned Space",boston-massachusetts,"[accessory dwelling unit, accessory dwelling]"
3,Setting up bank account for rental,charlotte-north-carolina,[ADU]
4,Looking for Advice on Structuring a Fair Real ...,fort-collins-colorado,[ADU]


In [46]:
# pandas display width change to 1000
pd.set_option('display.max_colwidth', 200)
adu_df 


,location,title,url,topic_id,list_scraped_at,content,author,author_url,post_date,view_count,vote_count,comment_count,replies,scraped_at,total_replies,scraped_from_city,matched_keywords
0,boston-massachusetts,Selling Low Interest Rate Investment Property to Scale?,https://www.biggerpockets.com/forums/48/topics/1252006-selling-low-interest-rate-investment-property-to-scale,1252006,2025-08-20T09:33:16.200325,"Hey BP community hope everyone is having a good year, I'm hoping to get others perspective on what they would do with a rental property in today's market. The property is performing well at a low ...",German Fernandez,https://www.biggerpockets.com/users/germanf3,2025-07-21T15:15:41-07:00,37,0,0,[],2025-08-20T09:35:15.118691,0,boston-massachusetts,[ADU]
1,boston-massachusetts,Learn About ADUs in Massachusetts,https://www.biggerpockets.com/forums/521/topics/1240758-learn-about-adus-in-massachusetts,1240758,2025-08-20T09:33:16.200395,"https://www.yourmaadu.com/Hi BP community!Did you know ADUs are now allowed BY RIGHT in Massachusetts? I'm hosting a free, online webinar offering the most recent information on the new ADU law, b...",Tyler Munroe,https://www.biggerpockets.com/users/tylerm154,2025-04-25T06:11:18-07:00,38,1,1,"[{'id': 'https://www.biggerpockets.com/forums/521/topics/1240758-learn-about-adus-in-massachusetts#post_7066026', 'author': 'James McGovern', 'author_url': 'https://www.biggerpockets.com/users/jam...",2025-08-20T09:35:55.261554,1,boston-massachusetts,"[ADU, ADUs]"
2,boston-massachusetts,"How to Rent out ""Mixed-Use"" Zoned Space",https://www.biggerpockets.com/forums/52/topics/1247045-how-to-rent-out-mixed-use-zoned-space,1247045,2025-08-20T09:33:23.492292,"Hi, I am an investor in the south shore of Mass and currently have a duplex property that is zoned in a ""Mixed-Use"" zone. I am currently renting out both units but I also have an attached 1-2 car ...",Alex C.,https://www.biggerpockets.com/users/acon11,2025-06-12T07:13:20-07:00,13,0,1,"[{'id': 'https://www.biggerpockets.com/forums/52/topics/1247045-how-to-rent-out-mixed-use-zoned-space#post_7049902', 'author': 'Jarrett Lau', 'author_url': 'https://www.biggerpockets.com/users/knl...",2025-08-20T09:38:08.837839,1,boston-massachusetts,"[accessory dwelling unit, accessory dwelling]"
3,charlotte-north-carolina,Setting up bank account for rental,https://www.biggerpockets.com/forums/12/topics/1256613-setting-up-bank-account-for-rental,1256613,2025-08-20T09:49:27.156141,"Looking to setup a bank account for my adu. What banks do you use and why, also do i need a account for rent and then a separate account for security deposit? When i return the security deposit do...",Braeden Warg,https://www.biggerpockets.com/users/braedenw3,2025-08-05T06:54:43-07:00,40,1,1,"[{'id': 'https://www.biggerpockets.com/forums/12/topics/1256613-setting-up-bank-account-for-rental#post_7083563', 'author': 'Ned Carey', 'author_url': 'https://www.biggerpockets.com/users/ncarey',...",2025-08-20T09:51:21.881929,1,charlotte-north-carolina,[ADU]
4,fort-collins-colorado,Looking for Advice on Structuring a Fair Real Estate Partnership,https://www.biggerpockets.com/forums/48/topics/1249896-looking-for-advice-on-structuring-a-fair-real-estate-partnership,1249896,2025-08-20T10:23:13.067148,"Hey everyone, I’m putting together a real estate partnership and would really appreciate some input on how to structure it fairly. Here’s the situation: We’re looking at purchasing a duplex that I...",Austin Alvarado,https://www.biggerpockets.com/users/austina244,2025-07-06T15:04:00-07:00,71,1,6,"[{'id': 'https://www.biggerpockets.com/forums/48/topics/1249896-looking-for-advice-on-structuring-a-fair-real-estate-partnership#post_7061794', 'author': 'Dan Hawtree', 'author_url': 'https://www....",2025-08-20T10:24:08.627727,6,fort-collins-colorado,[ADU]
5,fort-worth-texas,Home & income producing rv spaces,https://www.biggerpockets.com/forums/311/topics/1246099-home-and-income-producing-rv-spaces,1246099,2025-08-20T10